### Run this within dea-notebooks/ on the VDI

In [ ]:
import sys
sys.path.append('dea-notebooks/Scripts/') # i.e. dea-notebooks/Scripts/
import datacube
from dea_datahandling import load_ard
import xarray as xr
import numpy as np
import pandas as pd
import numpy as np
import rasterio

dc = datacube.Datacube(app='sentinel_fmc')

1. Load raster of study area and the look-up-table for FMC calculation (520,922,003 20x20m pixels equalling 208,369 km2, and 12 column x 4400 row dataframe respectively)
2. Get datacube of Sentinel reflectance like the raster, for whole Sentinel timeseries (up to 2100 timestamps). Mask out raster koala_trees != 1 during or after getting reflectances? 
3. Calculate FMC using numpy with reflectance and LUT inputs
4. Store datacube of only FMC variable

In [ ]:
# Load LUT and select only relevant land cover ie forest
file_path = '/g/data/xc0/user/IvanK/field site_fmc_estimation/'

# read Look Up Table from file
df = pd.read_csv(file_path+'LUT_S2.csv', index_col='ID')
df = df.drop(columns=['lai','soil','n','443','490','1375','945'])
df.columns = ['fmc','landcover','green','red','red_edge1',
              'red_edge2','red_edge3','nir1','nir2','swir2','swir3']
df['ndii'] = (df['nir1']-df['swir2'])/(df['nir1']+df['swir2'])
# select just forest values
df = df.loc[df.landcover == "forest",:]
# create array in format
lut = df[["fmc","green","red","red_edge1","red_edge2","red_edge3",
          "nir1","nir2","swir2","swir3","ndii"]].values
# the squares of the LUT
squares = np.einsum("ij,ij->i", lut[:, 1:], lut[:, 1:]) ** 0.5

In [ ]:
# Load mask of vegetation which has KTI of 3 or more, including only the koala distribution (from gov listing), which is reprojected to Albers/20m
koala_trees = xr.open_rasterio('/g/data/xc0/user/IvanK/koala_area_variables/KTSI_NSW_5m_lam_VAO_INT_koalaAOI_compr.tif',
                              chunks={'y': 2576,'x': 4259}).drop('band').squeeze('band')
koala_trees.attrs['crs'] = 'EPSG:3577'

# Create time dimension which includes every day since 2015 (ie whole Sentinel data period)
time = pd.date_range("2021-04-01", freq="D", periods= 1 * 365)

# Add time dimension to kti data mask, in a new xr dataset
koala_trees = xr.Dataset({'koala_trees': koala_trees, 'time': time})

In [ ]:
# Query the datacube using the 'like' attribute of load_ard, to take the spatial and temporal resolution of the kti dataset
s2_cube = load_ard(dc=dc, products=['s2a_ard_granule','s2b_ard_granule'],
                 like=koala_trees, measurements=["nbart_blue","nbart_green","nbart_red",
                         "nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
                         "nbart_nir_1","nbart_nir_2","nbart_swir_2","nbart_swir_3"
                        ], mask_pixel_quality=True,
                 group_by='solar_day', dask_chunks = {'y': 2576,'x': 4259})

In [ ]:
# Mask the s2_cube using the 2d koala_trees data
s2_cube = s2_cube.where(koala_trees.koala_trees == 1)

In [ ]:
## start FMC model

refl = s2_cube[["nbart_green","nbart_red","nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
         "nbart_nir_1","nbart_nir_2","nbart_swir_2","nbart_swir_3"]].to_array().values/10000

s2_cube['ndvi'] = ((s2_cube.nbart_nir_1-s2_cube.nbart_red)/(s2_cube.nbart_nir_1+s2_cube.nbart_red)) 
s2_cube['ndii'] = ((s2_cube.nbart_nir_1-s2_cube.nbart_swir_2)/(s2_cube.nbart_nir_1+s2_cube.nbart_swir_2))
refl = np.concatenate([refl,s2_cube['ndii'].values[None,:,:]], axis=0)

# create array frame
canvas = np.ones(s2_cube['ndvi'].values.shape, dtype=np.float32) * np.nan
# number of LUT values to consider in retrieval
top_n = 150

for t in range(s2_cube['ndvi'].values.shape[0]):

    for j in range(s2_cube['ndvi'].values.shape[1]):

        for i in range(s2_cube['ndvi'].values.shape[2]):
            x = refl[:,t, j, i]
#             m = forest_msk[j, i]
            if np.isnan(s2_cube['ndvi'][t, j, i]) or s2_cube['ndvi'][t, j, i] < 0.15:
#             if m != 1 or s2_cube['ndvi'][t, j, i] < 0.15: # already masked
                continue

            θ = -1 * (
                np.einsum("ij,j->i", lut[:, 1:], x)
                / (np.einsum("i,i->", x, x) ** 0.5 * squares)
            )

            idxs = np.argpartition(θ, top_n)[:top_n]
            canvas[t, j, i] = np.median(lut[idxs, 0])
s2_cube['FMC'] = (['time','y','x'], canvas) # TODO error: calculates FMC for pixels which have been masked in data retrieval(e.g clouds), so it is calculating FMC for nan in refl
s2_cube = s2_cube.drop_vars(["nbart_green","nbart_red","nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
         "nbart_nir_1","nbart_nir_2","nbart_swir_2","nbart_swir_3"])

In [ ]:
s2_cube.to_netcdf('/g/data/xc0/user/IvanK/koala_area_variables/s2_FMC_koala_timeseries.nc')